Figures for

# Predicting competition results from growth curves

by Yoav Ram, Eynat Deluss-Gur, Uri Obolski, Maayan Bibi, Judith Berman, and Lilach Hadany.

*A manuscript in preparation.*

#### Preprint: 

Ram et al. (2015) Predicting competition results from growth curves. bioRxiv doi: [10.1101/022640](http://biorxiv.org/content/early/2015/07/23/022640).

#### Notebook
This Jupyter notebook is an electronic supplementry material of the article.

The notebook **will be available** available at https://github.com/yoavram/curveball_ms/blob/master/supp.ipynb.

In [1]:
from imp import reload
import pkg_resources
import os
%matplotlib inline
import matplotlib.pyplot as plt
plt.warnings.simplefilter('ignore', FutureWarning)
plt.warnings.simplefilter('ignore', DeprecationWarning)
plt.rcParams['mathtext.default'] = 'regular'
import numpy as np
import pandas as pd
from scipy.stats import linregress
from scipy.integrate import odeint
import seaborn as sns
sns.set(style='white', context='paper', font_scale=2, palette='Set1')
red, blue, green = sns.color_palette('Set1', 3)
width, height = plt.rcParams['figure.figsize']

In [2]:
import curveball
import curveball.scripts.cli
print('Curveball, ', curveball.__version__)
folder = os.path.join('D:\\', 'Dropbox', 'ex silico')
print('Working folder:', folder)

Curveball,  0.2.3+7.g20f23ea.dirty
Working folder: D:\Dropbox\ex silico


In [3]:
def fig_xlabel(label):
    fig.text(0.5, 0, label, horizontalalignment='center', verticalalignment='bottom')

In [ ]:
import collections
datasets = collections.OrderedDict()
datasets['2015-11-18'] = {
    'plate_file': os.path.join(folder, 'tecan', 'plate_181115.csv'),
    'OD_file': os.path.join(folder, 'tecan', 'Yoav_181115.xlsx'),
    'max_time': 12,
    'bad_wells': ['B2','E1','C4','A2'] + # G outliers
                 ['H9','C11','A10','C12','A11','A9','C9','H11'] + # R outliers
                 ['E5','E8','H6','H8','H7','H5'], # RG outliers
    'flow_mean_file': os.path.join(folder, 'flow', 'mean_df_2015-11-18.csv'),
    'flow_std_file': os.path.join(folder, 'flow', 'std_df_2015-11-18.csv'),
    'lag': True
}
datasets['2015-12-14'] = {
    'plate_file': os.path.join(folder, 'tecan', 'plate_141215.csv'),
    'OD_file': os.path.join(folder, 'tecan', 'Yoav_141215.xlsx'),
    'max_time': 7,
    'bad_wells': ['{}{}'.format(c, i) for c in 'ABCDEFGH' for i in range(4, 8)] + # used for flow samples
                 ['B10','B11','C10','A11','E11','A12'] + # red outliers
                 ['D8','H8'], # RG outliers
    'flow_mean_file': os.path.join(folder, 'flow', 'mean_df_2015-12-14.csv'),
    'flow_std_file': os.path.join(folder, 'flow', 'std_df_2015-12-14.csv'),
    'lag': False
}
datasets['2015-12-23'] = {
    'plate_file': os.path.join(folder, 'tecan', 'plate_231215.csv'),
    'OD_file': os.path.join(folder, 'tecan', 'Yoav_231215.xlsx'),
    'max_time': 8,
    'bad_wells': ['{}{}'.format(c, i) for c in 'ABCDEFGH' for i in range(4, 8)] + # used for flow samples
                 ['A1'] + # G outliers
                 ['D11','A11','A12','D10','H10','C10'] + # R outliers
                 ['A8','A9', 'B8', 'B9', 'C9', 'D8', 'H8'], # RG outliers
    'flow_mean_file': os.path.join(folder, 'flow', 'mean_df_2015-12-23.csv'),
    'flow_std_file': os.path.join(folder, 'flow', 'std_df_2015-12-23.csv'),
    'lag': False
}

In [ ]:
for ds in datasets.values():
    ds['plate'] = pd.read_csv(ds['plate_file'])
    ds['df'] = curveball.ioutils.read_tecan_xlsx(ds['OD_file'], plate=ds['plate'], max_time=ds['max_time'])
    ds['df'] = ds['df'][~ds['df'].Well.isin(ds['bad_wells'])]
    ds['dfG'] = ds['df'][ds['df'].Strain.str.contains('GFP')]
    ds['dfR'] = ds['df'][ds['df'].Strain.str.contains('RFP')]
    ds['dfRG'] = ds['df'][ds['df'].Strain=='mixed']
    ds['RG0_mean'] = ds['dfRG'][(ds['dfRG'].Time==ds['dfRG'].Time.min())].OD.mean()
    ds['RG_time'] = ds['dfRG'].Time.unique()
    ds['RG_mean'] = ds['dfRG'].groupby(by='Time').OD.mean()
    ds['RG_std'] = ds['dfRG'].groupby(by='Time').OD.std(ddof=1)

In [ ]:
cols = len(datasets)
fig, ax = plt.subplots(1, cols, sharex=False, sharey=True, figsize=(width * cols, height))
for i,(name, ds) in enumerate(datasets.items()):
    curveball.plots.tsplot(ds['df'], ax=ax[i])
    ax[i].set(title=name, xlabel='')
    if i == 0:
        ax[i].set(ylabel=r'Density ($OD_{600}$)')        
    else:
        ax[i].set(ylabel='')
        ax[i].legend().set_visible(False)
fig_xlabel('Time (hr)')
#fig.savefig

In [ ]:
for ds in datasets.values():
    param_fix = {'y0', 'K'}
    param_guess = None
    if not ds['lag']:
        param_fix.add('q0')
        param_fix.add('v')
        param_guess = {'q0': np.inf, 'v': np.inf}
    models_G = curveball.models.fit_model(ds['dfG'], param_fix=param_fix, param_guess=param_guess, PRINT=False, PLOT=False)
    ds['model_G'] = models_G[0]
    models_R = curveball.models.fit_model(ds['dfR'], param_fix=param_fix, param_guess=param_guess, PRINT=False, PLOT=False)
    ds['model_R'] = models_R[0]
    models_RG = curveball.models.fit_model(ds['dfRG'], param_fix=param_fix, param_guess=param_guess, PRINT=False, PLOT=False)
    ds['model_RG'] = models_RG[0]

In [ ]:
cols = len(datasets)
fig, ax = plt.subplots(1, cols, sharex=False, sharey=True, figsize=(width * cols, height))
for i,(name, ds) in enumerate(datasets.items()):
    ds['model_G'].plot_fit(ax=ax[i], fit_kws={'color':green}, data_kws={'color':green,'alpha':0.5, 'marker':'.'}, init_kws={'ls':''})
    ds['model_R'].plot_fit(ax=ax[i], fit_kws={'color':red}, data_kws={'color':red, 'alpha':0.5, 'marker':'.'}, init_kws={'ls':''})
    ds['model_RG'].plot_fit(ax=ax[i], fit_kws={'color':blue}, data_kws={'color':blue,'alpha':0.5, 'marker':'.'}, init_kws={'ls':''})
    
    ax[i].set(title=name, xlabel='')
    if i == 0:
        ax[i].set(ylabel=r'Density ($OD_{600}$)')
    else:
        ax[i].set(ylabel='')
    ax[i].legend().set_visible(False)
fig_xlabel('Time (hr)')
fig.tight_layout()
sns.despine()

## Exponential model

In [ ]:
from statsmodels.nonparametric.smoothers_lowess import lowess

def smooth(x, y, **kwargs):
    if 'return_sorted' not in kwargs:
        kwargs['return_sorted'] = False
    if 'missing' not in kwargs:
        kwargs['missing'] = 'raise'
    yhat = lowess(y, x, **kwargs)
    return yhat

In [ ]:
def expnential_model(mGreen, mRed, frac_green=0.3, frac_red=0.3, di=2, 
                     colors=sns.color_palette('Set1', 3)[::2], ax=None, PLOT=False):
    G = np.unique(mGreen.best_fit)
    tG = np.unique(mGreen.userkws['t'])
    dt = (tG.max() - tG.min())/G.size
    dGdt = np.gradient(G, dt)

    R = np.unique(mRed.best_fit)
    tR = np.unique(mRed.userkws['t'])
    dt = (tR.max() - tR.min())/R.size
    dRdt = np.gradient(R, dt)

    dGdt_smooth = smooth(tG, dGdt, frac=frac_green)
    dRdt_smooth = smooth(tR, dRdt, frac=frac_red)
    
    
    imaxG = dGdt_smooth.argmax()
    slopeG, interceptG, _, _, _ = linregress(tG[imaxG-di:imaxG+di], np.log(G[imaxG-di:imaxG+di]))
    imaxR = dRdt_smooth.argmax()
    slopeR, interceptR, _, _, _ = linregress(tR[imaxR-di:imaxR+di], np.log(R[imaxR-di:imaxR+di]))

    if PLOT:
        if ax is None:
            fig, ax = plt.subplots()
        else:
            fig = ax.figure
        ax.plot(tG, G, color=colors[1])
        ax.plot(tG, np.exp(slopeG * tG + interceptG), '--', color=colors[1])
        ax.plot(tR, R, color=colors[0])
        ax.plot(tR, np.exp(slopeR * tR + interceptR), '--', color=colors[0])

        ax.set(ylim=(0.9*min(G.min(), R.min()), 1.1*max(G.max(), R.max())), xlabel='Time (hour)', ylabel='log OD', yscale='log')
        ax.text(0.5, 0.8*R.max(), ('$r_G={:.2g}$'+'\n'+'$r_R={:.2g}$').format(slopeG, slopeR))
        fig.tight_layout()
        sns.despine()
        return slopeG, interceptG, slopeR, interceptR, fig, ax 
    return slopeG, interceptG, slopeR, interceptR

In [ ]:
cols = len(datasets)
fig, ax = plt.subplots(1, cols, sharex=False, sharey=False, figsize=(width * cols, height))
for i,(name, ds) in enumerate(datasets.items()):
    ds['slopeG'], ds['interceptG'], ds['slopeR'], ds['interceptR'], _, _  = expnential_model(ds['model_G'], ds['model_R'], ax=ax[i], PLOT=True)    
    ax[i].set(title=name, xlabel='')
    if i == 0:
        ax[i].set(ylabel=r'Log Density ($OD_{600}$)')
    else:
        ax[i].set(ylabel='')
fig_xlabel('Time (hr)')
fig.tight_layout()
sns.despine()

## Competition experiment

In [ ]:
for ds in datasets.values():
    ds['flow_mean_df'] = pd.read_csv(ds['flow_mean_file'])
    ds['flow_std_df'] = pd.read_csv(ds['flow_std_file'])

In [ ]:
reload(curveball.competitions)
ode = curveball.competitions.baranyi_roberts_yr

for ds in datasets.values():
    K = ds['model_G'].best_values['K'], ds['model_R'].best_values['K']
    r = ds['model_G'].best_values['r'], ds['model_R'].best_values['r']
    nu = ds['model_G'].best_values.get('nu',1), ds['model_R'].best_values.get('nu',1)
    q0 = ds['model_G'].best_values.get('q0', np.inf), ds['model_R'].best_values.get('q0', np.inf)
    v = ds['model_G'].best_values.get('v', r[0]), ds['model_R'].best_values.get('v', r[1])
    y0 = ds['model_G'].best_values['y0']/2, ds['model_R'].best_values['y0']/2

    f0 = np.array(ds['flow_mean_df'][ds['flow_mean_df'].time==ds['flow_mean_df'].time.min()].weight) # initial frequencies from flow data
    RG_model = np.unique(ds['model_RG'].best_fit) # expected initial mixed culture OD from mixed culture model
    RG_mean = ds['dfRG'].groupby('Time').OD.mean().as_matrix()
    RG_std = ds['dfRG'].groupby('Time').OD.std().as_matrix()
    RG_t = np.unique(ds['dfRG'].Time)
    G0, R0 = f0 * RG_model.min() # expected initial OD of each strain in mixed culture

    t, y, a = curveball.competitions.fit_and_compete(
        ds['model_G'], 
        ds['model_R'], 
        ds['dfRG'],
        ode=ode,
        y0=(G0, R0),
        aguess=(1,1),
        PLOT=False,
        fixed=False
    )
    MRSE = ((odeint(ode, (G0, R0), RG_t, args=(K, r, nu, q0, v, a)).sum(axis=1) - RG_mean)**2).mean()
    
    ds['RG_model'] = RG_model
    ds['RG_mean'] = RG_mean
    ds['RG_std'] = RG_std
    ds['RG_t'] = RG_t
    ds['predicted_t'] = t
    ds['predicted_y'] = y
    ds['MRSE'] = MRSE

In [ ]:
rows = 2#len(datasets)
cols = 2
fig, ax = plt.subplots(rows, cols, sharex=True, sharey=False, figsize=(width * cols, height * rows))
ax.resize((rows, cols))
for row,(name, ds) in enumerate(datasets.items()):
    if row >= rows:
        break

    
    RG_model = ds['RG_model']
    RG_mean = ds['RG_mean']
    RG_std = ds['RG_std']
    RG_t = ds['RG_t']
    t = ds['predicted_t']
    y = ds['predicted_y']
    MRSE = ds['MRSE']
    
    ysum = y.sum(axis=1)
    p1 = y[:, 0] / ysum
    p2 = y[:, 1] / ysum
    
    ax[row, 0].plot(t, y[:, 0], color=green)
    ax[row, 0].plot(t, y[:, 1], color=red)
    ax[row, 0].plot(t, ysum, color=blue)
    ax[row, 0].errorbar(RG_t, RG_mean, RG_std, fmt='o', color=blue)
    
    ax[row, 0].set(
        xlabel='Time (hour)', 
        ylabel='OD', 
        xlim=(-0.5, t.max() + 0.5),
        ylim=(0, 1.1*RG_model.max()),
        title="MRSE: {:.2g}".format(MRSE)
    )

    ax[row, 1].plot(t, p1, color=green)
    ax[row, 1].plot(t, p2, color=red)
    idx = ds['flow_mean_df'].Strain == 'Green'
    g0 = ds['flow_mean_df'][idx].weight.max()
    ax[row, 1].errorbar(
        ds['flow_mean_df'][idx].time, 
        ds['flow_mean_df'][idx].weight.as_matrix(), 
        ds['flow_std_df'][idx].weight.as_matrix(), 
        marker='o', 
        ls='', 
        color=green
    )
    idx = ds['flow_mean_df'].Strain == 'Red'
    r0 = ds['flow_mean_df'][idx].weight.min()
    ax[row, 1].errorbar(
        ds['flow_mean_df'][idx].time, 
        ds['flow_mean_df'][idx].weight.as_matrix(), 
        ds['flow_std_df'][idx].weight.as_matrix(), 
        marker='o', 
        ls='', 
        color=red
    )    
    ax[row, 1].set(
        xlabel='Time (hour)', 
        ylabel='Frequency',
        #xlim=(-0.5, t.max() + 0.5), 
        xlim=(0, ds['flow_mean_df'][idx].time.max() + 0.5),
        ylim=(0, 1),
        title="a1={:.2g}, a2={:.2g}".format(*a)
    )

sns.despine()
fig.tight_layout()
# Nexp = G0*np.exp(slopeG * t), R0*np.exp(slopeR * t)
# Bexp = Nexp[0] + Nexp[1]
# pGexp = Nexp[0]/Bexp
# pRexp = Nexp[1]/Bexp
# ax[0].plot(t, Bexp, 'k--')
# ax[1].plot(t, pGexp, '--k')
# ax[1].plot(t, pRexp, '--k')
